In [ ]:
import pandas as pd
import unicodedata, re
import numpy as np


In [ ]:
data = pd.read_csv('../data/raw/Operace_anonymizovano.csv', delimiter=';')

In [ ]:
data.head(5)

In [ ]:
data.iloc[0]

In [ ]:
obj_cols = data.select_dtypes(include="object").columns
def _maybe_to_numeric(s: pd.Series) -> pd.Series:
    frac = s.str.replace(r"\s", "", regex=True).str.replace(",", ".", regex=False)
    num = pd.to_numeric(frac, errors="coerce")
    return num if num.notna().mean() > 0.5 else s
data[obj_cols] = data[obj_cols].apply(_maybe_to_numeric)

In [ ]:
ts_columns = [
    "DATUM",
    "DATUM_CAS_OD",
    "DATUM_CAS_DO",
    "DATUM_CAS_PREV",
    "DATUM_CAS_PREV_ORIGINAL"
]
for c in ts_columns:
    dt = pd.to_datetime(data[c], errors="coerce")
    if dt.notna().sum() > len(data)*0.5:
        if dt.dt.tz is None:
            dt = dt.dt.tz_localize("CET")
        data[c] = dt

In [ ]:
def slug(s):
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    s = re.sub(r'[^A-Za-z0-9_]+', '_', s).strip('_')
    return s.lower()

data.columns = [slug(c) for c in data.columns]

In [ ]:
data["total_time"] = data[["minut_pick", "minut_mezipick", "minut_nopick"]].sum(axis=1, skipna=True)
data["pick_share"] = data["minut_pick"] / data["total_time"]
data["mezipick_share"] = data["minut_mezipick"] / data["total_time"]
data["nopick_share"] = data["minut_nopick"] / data["total_time"]
data["handle_time_s"] = (data["datum_cas_do"] - data["datum_cas_od"]).dt.total_seconds()


In [ ]:
missingness = (
    pd.DataFrame({
        "dtype": data.dtypes.astype(str),
        "non_null": data.notna().sum(),
        "nulls": data.isna().sum(),
    })
    .assign(null_pct=lambda t: (t["nulls"] / len(data) * 100).round(2))
    .sort_values("null_pct", ascending=False)
)
missingness

In [ ]:
# In this section we are looking for suspicious rows based on several rules
# We create boolean flags for each rule and then summarize the results

flags = pd.DataFrame(index=data.index)
quantity_col = "mnozstvi"
start_col = "datum_cas_od"
end_col = "datum_cas_do"

# -------------------- 1) total_time == 0 while quantity > 0 ---------------------------------------
if quantity_col:
    flags["zero_speed_pos_quantity"] = (data["total_time"].fillna(0).eq(0)) & (data[quantity_col].fillna(0) > 0)
else:
    flags["zero_speed_pos_quantity"] = False

# -------------------- 2) End before start OR negative durations ---------------------------------------
flags["end_before_start"] = data[end_col] < data[start_col]
flags["negative_duration"] = data["handle_time_s"] < 0


flags["any_suspicious"] = flags.any(axis=1)

In [ ]:
suspicious_rows = data.loc[flags["any_suspicious"]].copy()
cols_preview = [c for c in ['cinnost_id', quantity_col, "total_time", "minut_pick", "minut_mezipick", "minut_nopick", start_col, end_col, "handle_time_s"]]
suspicious_preview = suspicious_rows[cols_preview].join(flags.loc[suspicious_rows.index])
summary_counts = flags.sum().sort_values(ascending=False)

In [ ]:
print("-------- Missingness (top 5 by null %) --------")
display(missingness.loc[["total_time", "minut_pick", "minut_mezipick", "minut_nopick"]])

print("\n-------- Suspicious flags counts --------")
print(summary_counts)

print("\n-------- Suspicious rows (preview) --------")
display(suspicious_preview.head(5))

In [ ]:
cleaned = data[~flags["any_suspicious"]].copy()
print(f"Original rows: {len(data)}, cleaned rows: {len(cleaned)}")

# Outliers

## Distribution and tails

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="total_time", nbins=100, title="Distribution of total_time")
fig.update_layout(bargap=0.1)
fig.add_vline(x=data["total_time"].quantile(0.95), line_dash
="dash", line_color="green", annotation_text="95th percentile", annotation_position="top left")
fig.add_vline(x=data["total_time"].quantile(0.99), line_dash="dash", line_color="red", annotation_text="99th percentile", annotation_position="top left")
fig.show()


In [ ]:
s = cleaned["total_time"].dropna().astype(float)
q1, q3 = s.quantile([0.25, 0.75])
p95, p99 = s.quantile([0.95, 0.99])
iqr = max(q3 - q1, 1e-12)
tail_gap = max(p99 - p95, 1e-12)

print(f"total_time stats (cleaned data):")
print(f"  Count: {len(s)}")
print(f"  Mean: {s.mean():.2f}, Std: {s.std():.2f}")
print(f"  Min: {s.min():.2f}, Max: {s.max():.2f}")
print(f"  25th percentile (Q1): {q1:.2f}")
print(f"  75th percentile (Q3): {q3:.2f}")
print(f"  IQR: {iqr:.2f}")
print(f"  95th percentile: {p95:.2f}")
print(f"  99th percentile: {p99:.2f}")
print(f"  Tail gap (P99 - P95): {tail_gap:.2f}")

### ICQ

In [ ]:
IQR_K  = 3.0

thr_iqr_work  = q3  + IQR_K  * iqr
thr_iqr_raw  = float(thr_iqr_work)

mask_rule = cleaned["total_time"] > thr_iqr_work
print(f"IQR fence (raw)       : {thr_iqr_raw:.3f}")

### Z-score method

In [ ]:
Z = 3
med = float(np.median(s))
mad = float(np.median(np.abs(s - med))) or 1e-12
zs = 0.6745 * (s - med) / mad

mask_z = pd.Series(False, index=cleaned.index)
mask_z.loc[s.index] = zs > Z

thr_z_work = med + (Z / 0.6745) * mad
thr_z_raw  = float(thr_z_work)

In [ ]:
cleaned["outlier_rule"] = mask_rule
cleaned["outlier_z"] = mask_z

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import math

s_all = cleaned["total_time"].astype(float)
fig = go.Figure()
fig.add_trace(go.Histogram(x=s_all, nbinsx=80, name="total_time", opacity=0.85))

for name, thr in [("IQR", thr_iqr_raw), (f"Robust Z={Z}", thr_z_raw)]:
    fig.add_trace(go.Scatter(
        x=[thr, thr],
        y=[0, max(1, math.ceil(len(s_all) / 10))],
        mode="lines",
        name=name,
        line=dict(width=2, dash="dash")
    ))

fig.update_layout(
    title="Distribution of total_time with outlier thresholds",
    xaxis_title="total_time (minutes)",
    yaxis_title="count",
    barmode="overlay",
    template="plotly_white"
)
fig.show()

In [ ]:
# 1) Zoomed histogram (0–12 min)
thr_rule = thr_iqr_raw
thr_z    = thr_z_raw

fig1 = go.Figure()
fig1.add_trace(go.Histogram(x=s, nbinsx=120, xbins=dict(start=0,end=12,size=0.1), name="total_time", opacity=0.85))
for name, thr in [("IQR", thr_rule), (f"Robust Z=3", thr_z)]:
    fig1.add_trace(go.Scatter(x=[thr, thr], y=[0, s.size/20], mode="lines", name=name, line=dict(width=2, dash="dash")))
fig1.update_layout(title="total_time (0–12 min zoom)", xaxis_title="minutes", yaxis_title="count", template="plotly_white")

# 2) Same histogram with log-y to expose the tail
fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=s, nbinsx=100, name="total_time", opacity=0.85))
for name, thr in [("IQR", thr_rule), (f"Robust Z=3", thr_z)]:
    fig2.add_trace(go.Scatter(x=[thr, thr], y=[0, s.size/20], mode="lines", name=name, line=dict(width=2, dash="dash")))
fig2.update_layout(title="total_time (log-y)", xaxis_title="minutes", yaxis_title="count", template="plotly_white")
fig2.update_yaxes(type="log")

# 3) ECDF (percentile view)
"""
for any x on the x-axis, the y-value is the fraction of rows with total_time ≤ x
"""
x = np.sort(s.values)
y = np.arange(1, len(x)+1) / len(x)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=x, y=y, mode="lines", name="ECDF"))
for name, thr in [("IQR", thr_rule), (f"Robust Z=3", thr_z)]:
    fig3.add_trace(go.Scatter(x=[thr, thr], y=[0,1], mode="lines", name=name, line=dict(width=2, dash="dash")))
fig3.update_layout(title="ECDF of total_time", xaxis_title="minutes", yaxis_title="cumulative fraction", template="plotly_white")

fig1.show(); fig2.show(); fig3.show()

In [ ]:
target_data = cleaned[~(cleaned["outlier_z"])].copy()
target_data.to_parquet("../data/interim/cleaned_total_time.parquet", index=False)